In [1]:
import os
import openslide

In [2]:
# NAS 접근방법

In [11]:
nas_path = "Z:\\"

In [13]:
print(os.listdir(os.path.join(nas_path, "data")))

['.DS_Store', 'clam_data', 'level0', 'level2', 'patch', 'practice', 'target_ln.csv', 'train', 'valid']


In [14]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from PIL import Image
import random
import openslide
import re
import math
from skimage.morphology import skeletonize
import xml.etree.ElementTree as ET
import tifffile
import skimage
from skimage import morphology
from mpl_toolkits.axes_grid1 import make_axes_locatable
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as trasnforms

import timm
import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2
import h5py

In [16]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    random.seed(seed_value) 
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars\n
        torch.backends.cudnn.deterministic = True  #needed\n
        torch.backends.cudnn.benchmark = False

seed = 42
random_seed(seed,True)

In [17]:
psize = (224,224)
level_3 = 3
level_1 = 1
overlap = (192,192)
batch_size = 100
threshold = 0.8

In [18]:
model_path = {
    'level_3' : "Z:\\hsyoo\\pth\\pth_80\\level_3\\best_model_level3.pth",
    'level_1' : "hi"
}

In [19]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [20]:
val_imtrans = A.Compose([
    A.Normalize(),
    ToTensorV2()
])

In [22]:
def read_wsi_clf(slide, level, psize, overlap, net):
    max_x = slide.level_dimensions[level][0]
    max_y = slide.level_dimensions[level][1]
    num = 0
    
    
    out_fill_3 = np.zeros(psize)
    out_fill_2 = np.zeros((psize[0]//2,psize[1]//2))
    out_fill_1 = np.zeros((psize[0]//4, psize[1]//4))
    if level == 3 or level == 4:
        overlay = np.zeros([max_y, max_x])
        reference = np.zeros([max_y, max_x])
    elif level == 2:
        overlay = np.zeros([max_y//2, max_x//2])
        reference = np.zeros([max_y//2, max_x//2])
    elif level == 1:
        overlay = np.zeros([max_y//4, max_x//4])
        reference = np.zeros([max_y//4, max_x//4])

    steps_x = int(math.ceil((max_x - overlap[0]) / float(psize[0] - overlap[0])))
    steps_y = int(math.ceil((max_y - overlap[1]) / float(psize[1] - overlap[1])))
    start_coords = list()
    
    for y in tqdm(range(0,steps_y)):
        col_list = list()
        
        for x in range(0, steps_x):
            x_start = x*(psize[0] - overlap[0])
            x_end = x_start + psize[0]
            y_start = y*(psize[1] - overlap[1])
            y_end = y_start + psize[1]
            
            if (x_end > max_x):
                x_overlap = x_end - max_x
                x_start = max_x - psize[0]
                x_end = max_x
            if(y_end > max_y):
                y_start = max_y - psize[1]
                y_end = max_y
                
            temp = np.array(slide.read_region((x_start*(2**(level)), y_start*(2**(level))), level, psize))
            temp = temp[:,:,:3]
            temp = temp.astype(np.uint8)
            temp = val_imtrans(image = temp)['image'].unsqueeze(0).to(device).float()

            if level == 3 or level ==4:
                start_coords.append((y_start,x_start))
            elif level == 2:
                start_coords.append((y_start//2,x_start//2))
            elif level == 1:
                start_coords.append((y_start//4,x_start//4))
            
            """
            batch size로 쌓기
            """
            if num==0:
                cat = temp
            else:
                cat = torch.cat((cat,temp), dim = 0)
            num+=1
            
            """
            model에 배치로 넣기
            """
            if num == batch_size:
                out = net(cat)
                out = F.softmax(out, dim = 1)
                out = out.detach().cpu().numpy()
                out = out[:,1]
                if level == 3 or level == 4:
                    for i in range(batch_size):
                        out_fill_3.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] += out_fill_3
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] +=1
                        if i ==batch_size-1:
                            num=0
                            start_coords = []
                elif level == 2:
                    for i in range(batch_size):
                        out_fill_2.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] += out_fill_2
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] +=1
                        if i ==batch_size-1:
                            num=0
                            start_coords = []
                elif level == 1:
                    for i in range(batch_size):
                        out_fill_1.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] += out_fill_1
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] +=1
                        if i ==batch_size-1:
                            num=0
                            start_coords = []
            elif y==steps_y-1 and x==steps_x-1:
                out = net(cat)
                out = F.softmax(out, dim = 1)
                out = out.detach().cpu().numpy()
                out = out[:,1]
                if level == 3 or level == 4:
                    for i in range(num):
                        out_fill_3.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] += out_fill_3
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] +=1
                elif level ==2:
                    for i in range(num):
                        out_fill_2.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] += out_fill_2
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] +=1
                elif level ==1:
                    for i in range(num):
                        out_fill_1.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] += out_fill_1
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] +=1

    output = overlay / reference
    return output

In [24]:
net = timm.create_model("tf_efficientnet_b0_ns", pretrained = True, num_classes = 2)
net.load_state_dict(torch.load(model_path['level_3']))
net = net.to(device).eval()


# In[12]:


root_dir = "Z:\\data\\"
modes = ["valid"]
meta_labels = ["NOLN_metastasis", "LN_metastasis"]
wsi_fns = list()
include_num = []
for i in range(66,101):
    num = format(i, '03')
    include_num.append(num)
for mode in modes:
    for meta_label in meta_labels:
        for patient in sorted(os.listdir(os.path.join(root_dir, mode, meta_label))):
            if patient.split('.')[-1] == 'mrxs' and patient.split('.')[0].split('_')[1] in include_num:
                wsi_fns.append(os.path.join(root_dir, mode, meta_label, patient))

In [26]:
magnifications_40x = ["LN_068", "LN_069", "LN_070", "LN_071", "LN_072", "LN_073", "LN_074", "LN_075", "LN_076", "LN_077", 
"LN_078", "LN_079", "LN_080", "LN_081", "LN_082", "LN_083", "LN_084", "LN_085", "LN_086", "LN_087", 
"LN_088", "LN_089", "LN_090", "LN_091", "LN_092", "LN_093", "LN_094", "LN_095", "LN_096", "LN_097",
"LN_098",  "LN_099", "LN_100", "NOLN_067", "NOLN_068", "NOLN_069", "NOLN_070", "NOLN_071", "NOLN_072", "NOLN_073", "NOLN_074", 
"NOLN_075", "NOLN_076", "NOLN_077", "NOLN_078", "NOLN_079", "NOLN_080", "NOLN_081", "NOLN_082", 
"NOLN_083", "NOLN_084", "NOLN_085", "NOLN_086", "NOLN_087", "NOLN_088", "NOLN_089", "NOLN_090", 
"NOLN_091", "NOLN_092", "NOLN_093", "NOLN_094", "NOLN_095", "NOLN_096", "NOLN_097", "NOLN_098", 
"NOLN_099", "NOLN_100"]

In [ ]:
for wsi_fn in wsi_fns:
    name = wsi_fn.split('/')[-1].split('.')[0]
    print(name)
    if name in magnifications_40x:
        level_3 = 4
    else:
        level_3 = 3
    
#     f = h5py.File(f'/mnt/pathology/hsyoo/h5/heatmap/{name}.h5', 'w')
    slide = openslide.OpenSlide(wsi_fn)
    print(f"slide.level_dimensions : {slide.level_dimensions}, apply level : {level_3}")
    overlay_3 = read_wsi_clf(slide, level_3, psize, overlap, net)
    print(f"overlay_3 shape : {overlay_3.shape}")
    print(f"overlay_3 min max : {overlay_3.min()}, {overlay_3.max()}")
    f['level_3'] = overlay_3
    
    """
    여기 위에 overlay추가 가능, 밑부터는 저장단계
    """
    d=copy.deepcopy(overlay_3)
    overlay_3[overlay_3 >= threshold] =1
    overlay_3[overlay_3 < threshold] = 0
    overlay_3 = overlay_3.astype(bool)
    b = morphology.remove_small_objects(overlay_3, 50000, connectivity=8)
    c = morphology.remove_small_holes(b, 50000, connectivity=8)
    slide = np.array(slide.read_region((0,0),level_3,slide.level_dimensions[level_3]))[:,:,:3]
    
    f['post'] = c # post-processing
    """
    visualization
    """
    fig = plt.figure(dpi = 200)
    ax = fig.add_subplot(1,4,1)
    plt.imshow(slide)
    plt.axis('off')
    plt.title(f"{name}")


    ax = fig.add_subplot(1,4,2)
    plt.imshow(slide)
    plt.imshow(overlay_3,cmap='jet', alpha = 0.5)
    plt.title('Prediction')
    plt.axis('off')
    
    ax = fig.add_subplot(1,4,3)
    plt.imshow(slide)
    plt.imshow(c,cmap='jet', alpha = 0.5)
    plt.axis('off')
    plt.title('Post-process')

    ax = fig.add_subplot(1,4,4)
    im = plt.imshow(d, vmin = 0.5, vmax = 1.0, cmap = 'jet')
    plt.title('heatmap')
    plt.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    plt.tight_layout()
    plt.show()
#     fig.savefig(f"/mnt/pathology/hsyoo/step1_predict/heatmap_80/{name}_l3.png")
    plt.close(fig)
    """
    위까지는 저장단계, 밑부터는 후처리 단계! -> 후처리는 bool형태로 이루어져야하므로 Thresholding이 발생한다!
    자세한 후처리 사항들은 check_segmentation_80의 remove_small_object, remove_small_holes를 통해 진행하자.
    step2 aug짜는건 후처리 전 threshold값으로 미리 짜둘 수 있다. bool로 바꾸면 어차피 똑같
    """
    # ~~~
    f.close()